In [1]:
# import numpy as np
# from keras.callbacks import Callback
# from keras import backend as K
# import pandas as pd

#class_weight={0:3.3359,1:0.3368,2:3.0813,3:2.7868,4:0.7300,5:1.4757}


      ###### প্রতি টা ক্লাসের ওয়েইট ক্যাল্কুলেট হচ্ছে এখানে।  #############
def compute_weight(Y, classes):
    num_samples = len(Y)
    n_classes = len(classes)
    num_bin = np.bincount(Y[:, 0])
    class_weights = {i: (num_samples / (n_classes * num_bin[i])) for i in range(6)}
    return class_weights


def patientSplitter(randomIDfile,df2,split_portion):
    import pandas as pd
    
    df1 = pd.read_csv(randomIDfile,header=None)
    split_portion_numer=int(split_portion*61)
    
    train_pat_list = [int(each) for each in df1.iloc[:split_portion_numer].values]
    test_pat_list = [int(each) for each in df1.iloc[split_portion_numer:].values]
    print(test_pat_list)
    df3 = []
    df4 = []
    for pat_ID in train_pat_list:
        df3.append(df2[df2.patID == pat_ID].values)
        print(pat_ID)
    for pat_ID in test_pat_list:
        df4.append(df2[df2.patID == pat_ID].values) 
        print(pat_ID)
    del df2
    df3 = pd.np.vstack(df3)
    df4 = pd.np.vstack(df4)
    X_train = df3[0:, :3000]
    X_test= df4[0:, :3000]
    Y_train= df3[0:,3000:3001]
    Y_test= df4[0:,3000:3001]
    pat_train=df3[0:, 3002:3003]
    pat_test= df4[0:, 3002:3003]
    
    del pd
    return X_train,X_test,Y_train,Y_test,pat_train,pat_test	


    #### রেজাল্ট এর লগিং এখানে। একটা কল ব্যাক দিয়ে সিএসভি ফাইলে গিয়ে ডাম্প করে আসতে হবে।
def results_log(filepath, params):

    df = pd.read_csv(filepath)
    df1 = pd.read_csv() #training.csv

    #logging pipeline here ##

    #print()


    ########## লগ ম্যাট্রিক্স বানানো।  -_-  #################
class log_metrics():
    '''Custom callback to add new elements to epoch end logs and tensorflow graphs

        # Callback called on_epoch_end to calcualte validations set metrics using meta
        # information (Patient ID, channel ID, etc). End of epoch metrics are added to
        # `logs` which are later used by the tensorboard callback.
        # log_metrics should be specified in the callback list before tensorboard callback
        # '''
    

    ###### লগ ম্যাট্রিক্স ক্লাসের ইনিশিয়ালাইজার ##################
    #### চার টা জিনিস নিচ্ছে সে। এক্স এর মান, ওয়াই এর মান, পেশেন্ট আইডি। আর কাজ করানোর জন্য খরগোশ।
    def __init__(self, valX, valY, patID, **kwargs):
        self.valY = valY
        self.valX = valX
        self.patID = patID
        super(log_metrics,self).__init__(**kwargs) #### এই সুপার টা এই ফাংশনের পার্ট না হয়ে মেইন ক্লাসের পার্ট হয়ে গেল। 



    def on_epoch_end(self, epoch, logs):

        if logs is not None:
            predY = self.model.predict(self.valX, verbose=0)
            predY = np.argmax(predY, axis=-1)

            # Enter metric calculations per patient here
            #

            ### Learning Rate logging for Adam ###

            lr = self.model.optimizer.lr
            if self.model.optimizer.initial_decay > 0:
                lr *= (1. / (1. + self.model.optimizer.decay * K.cast(self.model.optimizer.iterations, K.dtype(self.model.optimizer.decay))))
            t = K.cast(self.model.optimizer.iterations, K.floatx()) + 1
            lr_t = lr * (K.sqrt(1. - K.pow(self.model.optimizer.beta_2, t)) / (1. - K.pow(self.model.optimizer.beta_1, t)))
            logs['lr'] = np.array(float(K.get_value(lr_t)))




In [2]:
from keras.layers import Conv1D, MaxPooling1D, Activation, add, Dropout, Input
from keras.layers.normalization import BatchNormalization
from keras import initializers
from keras.engine import Layer, InputSpec
#from keras.engine import input_layer
from keras import backend as K
from keras.constraints import max_norm
#from input_layer import Input

class Scale(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.

    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''

    def __init__(self, weights=None, axis=-1, momentum=0.9, beta_init='he_normal', gamma_init='he_normal', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma' % self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta' % self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


def res_subsam(input_tensor, filters=(64,64), kernel_size=16, subsam=2, dropout_rate=0.2, bias=False, maxnorm=4., **kwargs):
    eps = 1.1e-5
    nb_filter1, nb_filter2 = filters
    x = BatchNormalization(epsilon=eps, axis=-1)(input_tensor)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    x = Conv1D(filters=nb_filter1, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(x)  ##
    x = MaxPooling1D(pool_size=subsam)(x)
    x = BatchNormalization(epsilon=eps, axis=-1)(x)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    x = Conv1D(filters=nb_filter2, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(x)  ##
    short = Conv1D(filters=nb_filter2, kernel_size=kernel_size, padding='same', use_bias=bias,
                   kernel_constraint=max_norm(maxnorm), kernel_initializer=initializers.he_normal(seed=1))(
        input_tensor)  ##
    short = MaxPooling1D(pool_size=subsam)(short)
    x = add([x, short])
    return x


def res_nosub(input_tensor, filters=(64,64), kernel_size=16, dropout_rate=0.2, bias=False, maxnorm=4., **kwargs):
    eps = 1.1e-5
    nb_filter1, nb_filter2 = filters
    x = BatchNormalization(epsilon=eps, axis=-1)(input_tensor)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    x = Conv1D(filters=nb_filter1, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(x)  ##
    x = BatchNormalization(epsilon=eps, axis=-1)(x)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    x = Conv1D(filters=nb_filter2, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(x)  ##
    x = add([x, input_tensor])
    return x


def res_first(input_tensor, filters=(64,64), kernel_size=16, dropout_rate=0.2, bias=False, maxnorm=4., **kwargs):
    eps = 1.1e-5
    nb_filter1, nb_filter2 = filters
    x = Conv1D(filters=nb_filter1, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(input_tensor)  ##
    x = BatchNormalization(epsilon=eps, axis=-1)(x)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)
    x = Dropout(rate=dropout_rate, seed=1)(x)
    x = Conv1D(filters=nb_filter2, kernel_initializer=initializers.he_normal(seed=1), kernel_size=kernel_size,
               padding='same', use_bias=bias, kernel_constraint=max_norm(maxnorm))(x)  ##
    x = add([x, input_tensor])
    return x


def eegnet(eeg_length=3000, kernel_size=16, bias=False, maxnorm=4., **kwargs):

    '''
    Top model for the CNN
    Add details of module in docstring
        '''

    eps = 1.1e-5


    EEG_input = Input(shape=(eeg_length,1))
    x = Conv1D(filters=64, kernel_size=kernel_size, kernel_initializer=initializers.he_normal(seed=1), padding='same',
               use_bias=bias, kernel_constraint=max_norm(maxnorm))(EEG_input)  ##
    x = BatchNormalization(epsilon=eps, axis=-1)(x)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)  # ব্যাচ নরমালাইজ করা ভার্শন টা কে নিয়ে স্কেলের মধ্যে ঢুকানো 

    x = res_first(x, filters=[64, 64], kernel_size=kernel_size)
    x = res_subsam(x, filters=[64, 64], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[64, 64], kernel_size=kernel_size)
    x = res_subsam(x, filters=[64, 128], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[128, 128], kernel_size=kernel_size)
    x = res_subsam(x, filters=[128, 128], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[128, 128], kernel_size=kernel_size)
    x = res_subsam(x, filters=[128, 192], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[192, 192], kernel_size=kernel_size)
    x = res_subsam(x, filters=[192, 192], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[192, 192], kernel_size=kernel_size)
    x = res_subsam(x, filters=[192, 256], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[256, 256], kernel_size=kernel_size)
    x = res_subsam(x, filters=[256, 256], kernel_size=kernel_size, subsam=2)
    x = res_nosub(x, filters=[256, 256], kernel_size=kernel_size)
    x = res_subsam(x, filters=[256, 512], kernel_size=kernel_size, subsam=2)
    x = BatchNormalization(epsilon=eps, axis=-1)(x)
    x = Scale(axis=-1)(x)
    x = Activation('relu')(x)

    return x


C:\Users\User\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from __future__ import print_function, absolute_import, division
import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
# set_session(tf.Session(config=config))
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
from datetime import datetime
import argparse
import os
import tables
from keras.utils import to_categorical, plot_model
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.optimizers import Adamax as opt
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
import pandas as pd

from modules import *
from utils import *

if __name__ == '__main__':
	
	
	
	############ পারসার এই ক্যাচাল এইখান থেকে শুরু #######################
	
	
	########## পারসার ডেফিনিশন ########
	parser = argparse.ArgumentParser(description='')
	parser.add_argument("fold",
						help="csvfile to use")
	parser.add_argument("--seed", type=int,
						help="Random seed")
	parser.add_argument("--loadmodel",
						help="load previous model checkpoint for retraining (Enter absolute path)")
	parser.add_argument("--epochs", type=int,
						help="Number of epochs for training")
	parser.add_argument("--batch_size", type=int,
						help="number of minibatches to take during each backwardpass preferably multiple of 2")
	parser.add_argument("--verbose", type=int, choices=[1, 2],
						help="Verbosity mode. 1 = progress bar, 2 = one line per epoch (default 2)")
	parser.add_argument("--classweights", type=bool,
						help="if True, class weights are added")
	parser.add_argument("--comment",
						help="Add comments to the log files")

	args = parser.parse_args()
	print("%s selected" % (args.fold))
	foldname = args.fold

	###### পারসার থেকে নিয়ে ভ্যারিয়েবল গুলাতে মান বসানো, মান না থাকলে ডিফল্ট কত হবে সেইগুলাও বসান ########
	
	if args.seed:  # if random seed is specified
		print("Random seed specified as %d" % (args.seed))
		random_seed = args.seed
	else:
		random_seed = 1

	if args.loadmodel:  # If a previously trained model is loaded for retraining
		load_path = args.loadmodel  #### path to model to be loaded

		idx = load_path.find("weights")
		initial_epoch = int(load_path[idx + 8:idx + 8 + 4])

		print("%s model loaded\nInitial epoch is %d" % (args.loadmodel, initial_epoch))
	else:
		print("no model specified, using initializer to initialize weights")
		initial_epoch = 0
		load_path = False

	if args.epochs:  # if number of training epochs is specified
		print("Training for %d epochs" % (args.epochs))
		epochs = args.epochs
	else:
		epochs = 200
		print("Training for %d epochs" % (epochs))

	if args.batch_size:  # if batch_size is specified
		print("Training with %d samples per minibatch" % (args.batch_size))
		batch_size = args.batch_size
	else:
		batch_size = 1024
		print("Training with %d minibatches" % (batch_size))


	if args.verbose:
			verbose = args.verbose
			print("Verbosity level %d" % (verbose))
	else:
		verbose = 2
	if args.comment:
			comment = args.comment
	else:
		comment = None
	
	
	######## এই পর্যন্ত শুধু পারসার থেকে মান নিয়ে ভ্যারিয়েবল গুলা তে বসেছে। ###############
	
	### ডিরেক্টরি ডিফাইন করা জেনারেলাইজ করে ####
	
	model_dir = os.path.join(os.getcwd(),'..','models').replace('\\','/')
	fold_dir = os.path.join(os.getcwd(),'..','data').replace('\\','/')
	log_dir = os.path.join(os.getcwd(),'..','logs').replace('\\','/')
	log_name = foldname + '_' + str(datetime.now()).replace(' ','').replace('\\','/')
	print(os.path.join(model_dir,log_name))
	if not os.path.exists(os.path.join(model_dir,log_name)):
		new_dir =(os.path.join(os.getcwd(),'..','models',log_name)).replace('\\','/').replace(':','')
		print(new_dir)
		os.makedirs(new_dir)
	checkpoint_name = os.path.join(model_dir,log_name,'weights.{epoch04d}-{val_acc.4f}.hdf5'.replace(':','')) # make sure separate
																						# folder for each log_name
	results_file = os.path.join(os.getcwd().replace('\\','/'), '..', 'results.csv')
	##### ডিরেক্টরি ক্যাচাল শেষ #####
	
	
	##### প্যারামস হচ্ছে একটা লিস্ট যেটা নেটওয়ার্ক কে খাওয়াতে হবে। এই লিস্টে সব হাইপারপ্যারামিটার থেকে শুরু করে ফোল্ডারের নাম সব থাকবে। এটা শিখলাম। কাজ করানো টা শিখতে হবে ################
	
	params={               # still not universal

		'num_classes':7,  ### automate; number of classes depends on data fold
		'batch_size':batch_size,
		'epochs':epochs,
		'foldname':foldname,
		'random_seed':random_seed,
		'load_path':load_path,
		'shuffle':True,
		'initial_epoch':initial_epoch,
		'verbose':verbose,
		'eeg_length':3000,
		'kernel_size':16,
		'bias':True,
		'maxnorm':4.,
		'dropout_rate':0.5,
		'dropout_rate_dense':0.,
		'padding':'valid',
		'activation_function':'relu',
		'subsam':2,
		'trainable':True,
		'lr':.0001,
		'lr_decay':1e-5,

	}

	########### Data Prep ################

	# mat_cont = tables.open_file(os.path.join(fold_dir,foldname))

	#Elegant one
	#df2 = pd.read_csv( (os.path.join(fold_dir,foldname),header=None)
	#কামলা কাউন্টারপার্ট
	df2 = pd.read_csv('E:/SleepWell/ASSC-master/data/purifiedallDataChannel2.csv',header=None)
	df2.rename({3000: 'hyp', 3001: 'epoch', 3002: 'patID'}, axis="columns", inplace=True)


	#X = mat_cont['data']
	#Y = mat_cont['hyp']
	#patID = mat_cont['patID']

	
	#trainX, valX, trainY, valY = train_test_split(X, Y, test_size=0.2, random_state=random_seed) # patID split koro bacha
	trainX, valX, trainY, valY, pat_train, pat_val = patientSplitter('randomizedIDs.csv', df2, 0.7)
	
	#####এই স্প্লিটিং করা যাবে না। লিখতে হবে। পেশেন্ট আইডি এর উপর বেইজ করে স্প্লিট করতে হবে। 
	
	
	######পেশেন্ট আইডী বেজ করে স্প্লিট করা নিয়ে কাজ করতে হবে এইখানে। ১৫ জনের ডাটা যাবে ট্রেনিং এ, বাকি দের ডেটা যাবে ভ্যালিডেশনে। ##############
	trainY = to_categorical(trainY, params['num_classes'])
	valY = to_categorical(valY, params['num_classes'])

	########### Create Model ################
	###### মডেলের শুরুর দিকের লেয়ার গুলা মডুলস নামের পাই ফাইলে আছে। এখানে শুধু ডেন্স লেয়ার গুলা আলাদা করে জইন করা হচ্ছে, কারণ এইগুলা তেই চেঞ্জ আসবে।  ######
	##### হাইপারপ্যারামিটার গুলা ট্রেইন করার জন্য শুধু হাইপার প্যারামিটার গুলা সম্ভলিত অংশ আলাদা করে লেখা হচ্ছে। ##################
	#শিখলাম ব্যপারটা। #
	
	#প্রথমে ইইজি নেট টা কে তৈরি করা, সব প্যারামিটার তাকে বলে দিয়ে ###
	top_model = eegnet(**params)  # might have bugs; sub modules need kwargs integration
	#    top_model = eegnet(params)
	
	# এর পরে সেটার আউটপুট কে ফ্ল্যাটেন করা, যাতে করে ডেন্স লেয়ার এড করা যায়।  ফ্ল্যাটেন করার পরে প্রথম ডেন্স লেয়ার এড করা। আরো বেশি ডেন্স লেয়ার এড করা যেতে পারে ব্যপার টা তে। পরে চেষ্টা করে দেখতে হবে বিভিন্ন কনফিগারেশন।   ####
	#এখানে যেমন প্রথম ডেন্স লেয়ারের সাথেই সফটম্যাক্স করে আঊটপুট দেওয়া। এমন না করে আরেকটা ডেন্স লেয়ার রেখে সেইটা তে সফট্ম্যাএক্স করা উচিত। রান দেওার পরে সেই কাজ করতে হবে ###
	x = Flatten()(top_model)
	x = Dense(params['num_classes'], activation='softmax', kernel_initializer=initializers.he_normal(seed=random_seed),
			  kernel_constraint=max_norm(params['maxnorm']), use_bias=True)(x)  ##

	model = Model(top_model.Input, x) # এখানে দুইটা মডেল জোড়া লেগে যাচ্ছে। টপ মডেল, আর পরের ডেন্স করার পরের অংশ - এই দুইটা।
	model.summary() # মডেলের সামারি
	if load_path:  # If path for loading model was specified
		model.load_weights(filepath=load_path, by_name=False)
	plot_model(model, to_file='model.png', show_shapes=True) #  মডেল কে ইমেজ ফাইলে আঁকা
	model_json = model.to_json() #জেসন ফাইলে লেখা হচ্ছে মডেল টা কে। সব ধরনের প্রিকশন নিয়ে রাখা, আর কি।
	with open(os.path.join(model_dir,log_name,'model.json'), "w") as json_file:
		json_file.write(model_json)
	model.compile(optimizer=opt(**params), loss='categorical_crossentropy', metrics=['accuracy']) # মডেল কম্পাইলেশন। টেক্সটবুক আচরণ, অবশেষে

	####### Define Callbacks #######
	
	### ভ্যালিডেশন একুরেসির উপর বেজ করে চেজপয়েন্ট নিয়ে রাখা মডেল সেভ করার জন্য #########
	modelcheckpnt = ModelCheckpoint(filepath=checkpoint_name,
									monitor='val_acc', save_best_only=False, mode='max')
	### টেন্সরবোরড ইন্সট্যান্স কল করা ######								
	tensbd = TensorBoard(log_dir=os.path.join(log_dir,log_name),
						 batch_size=batch_size, histogram_freq=3,
						 write_grads=True,
						 # embeddings_freq=99,
						 # embeddings_layer_names=embedding_layer_names,
						 # embeddings_data=x_val,
						 # embeddings_metadata=metadata_file,
						 write_images=False)
						 
	##### সিএসভি লগারের ইন্সট্যান্স তৈরি করা, লগ সেইভ করার জন্য ###########					 
	csv_logger = CSVLogger(os.path.join(log_dir,log_name,'training.csv'))

	
	
	if args.classweights:
		params['class_weight'] = compute_weight(trainY, np.unique(trainY))
	else:
		params['class_weight'] = dict(zip(np.r_[0:params['num_classes']],np.ones(params['num_classes']))) # weighted 1

	####### Train #######

	try:

		model.fit(trainX, trainY,
				verbose=2,
				validation_data=(valX, valY),
				callbacks=[modelcheckpnt,
						 log_metrics(valX, valY, patID),
						 tensbd, csv_logger],
				  **params)  # might have bugs
		results_log(results_file,params)

	except KeyboardInterrupt:

		results_log(results_file, params)


usage: ipykernel_launcher.py [-h] [--seed SEED] [--loadmodel LOADMODEL]
                             [--epochs EPOCHS] [--batch_size BATCH_SIZE]
                             [--verbose {1,2}] [--classweights CLASSWEIGHTS]
                             [--comment COMMENT]
                             fold
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\User\Anaconda3\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
	model_dir = os.path.join(os.getcwd(),'..','models').replace('\\','/')
	fold_dir = os.path.join(os.getcwd(),'..','data').replace('\\','/')
	log_dir = os.path.join(os.getcwd(),'..','logs').replace('\\','/')
	log_name = foldname + '_' + str(datetime.now()).replace(' ','').replace('\\','/')
	print(os.path.join(model_dir,log_name))
	if not os.path.exists(os.path.join(model_dir,log_name)):
		new_dir =(os.path.join(os.getcwd(),'..','models',log_name)).replace('\\','/').replace(':','')
		print(new_dir)
		os.makedirs(new_dir)
	checkpoint_name = os.path.join(model_dir,log_name,'weights.{epoch04d}-{val_acc.4f}.hdf5'.replace(':','')) # make sure separate
																						# folder for each log_name
	results_file = os.path.join(os.getcwd().replace('\\','/'), '..', 'results.csv')
	##### ডিরেক্টরি ক্যাচাল শেষ #####
	
	
	##### প্যারামস হচ্ছে একটা লিস্ট যেটা নেটওয়ার্ক কে খাওয়াতে হবে। এই লিস্টে সব হাইপারপ্যারামিটার থেকে শুরু করে ফোল্ডারের নাম সব থাকবে। এটা শিখলাম। কাজ করানো টা শিখতে হবে ################
	
	params={               # still not universal

		'num_classes':6,  ### automate; number of classes depends on data fold
		'batch_size':1,
		'epochs':1,
		'foldname':"",
		'random_seed':7,
		'load_path':load_path,
		'shuffle':True,
		'initial_epoch':1,
		'verbose':1,
		'eeg_length':3000,
		'kernel_size':16,
		'bias':True,
		'maxnorm':4.,
		'dropout_rate':0.5,
		'dropout_rate_dense':0.,
		'padding':'valid',
		'activation_function':'relu',
		'subsam':2,
		'trainable':True,
		'lr':.0001,
		'lr_decay':1e-5,

	}

E:/_2018-12-1513:37:59.858444
E/_2018-12-15133759.858444


In [12]:
foldname= "E:/"
load_path = foldname

In [17]:
import pandas as pd
def patientSplitter(randomIDfile,df2,split_portion):
    import pandas as pd
    
    df1 = pd.read_csv(randomIDfile,header=None)
    split_portion_numer=int(split_portion*61)
    
    train_pat_list = [int(each) for each in df1.iloc[:split_portion_numer].values]
    test_pat_list = [int(each) for each in df1.iloc[split_portion_numer:].values]
    print(test_pat_list)
    df3 = []
    df4 = []
    for pat_ID in train_pat_list:
        df3.append(df2[df2.patID == pat_ID].values)
        print(pat_ID)
    for pat_ID in test_pat_list:
        df4.append(df2[df2.patID == pat_ID].values) 
        print(pat_ID)
    del df2
    df3 = pd.np.vstack(df3)
    df4 = pd.np.vstack(df4)
    X_train = df3[0:, :3000]
    X_test= df4[0:, :3000]
    Y_train= df3[0:,3000:3001]
    Y_test= df4[0:,3000:3001]
    pat_train=df3[0:, 3002:3003]
    pat_test= df4[0:, 3002:3003]
    
    del pd
    return X_train,X_test,Y_train,Y_test,pat_train,pat_test	



df2 = pd.read_csv('E:/SleepWell/ASSC-master/data/purifiedallDataChannel2.csv',header=None)
df2.rename({3000: 'hyp', 3001: 'epoch', 3002: 'patID'}, axis="columns", inplace=True)

	#trainX, valX, trainY, valY = train_test_split(X, Y, test_size=0.2, random_state=random_seed) # patID split koro bacha
trainX, valX, trainY, valY, pat_train, pat_val = patientSplitter('randomizedIDs.csv', df2, 0.6)

[36, 20, 44, 60, 59, 28, 14, 30, 11, 15, 29, 8, 48, 24, 40, 54, 16, 37, 10, 39, 7, 18, 42, 57, 31]
35
21
22
43
9
56
34
58
26
6
3
55
33
2
27
13
46
45
38
47
17
5
25
32
1
52
50
12
41
51
4
19
0
23
53
49
36
20
44
60
59
28
14
30
11
15
29
8
48
24
40
54
16
37
10
39
7
18
42
57
31


In [5]:
import numpy as np
np.shape(trainY)

(72236, 1)

In [18]:
num_samples = len(trainY)
n_classes = len(np.unique(trainY))
print(np.unique(trainY))
print(num_samples)
print(n_classes)
#num_bin = np.bincount(Y[:, 0])

[1. 2. 3. 4. 5. 6.]
72236
6


In [21]:

dummytrainY= trainY.astype(int)
dummytrainY= dummytrainY-1
num_bin = np.bincount(trainY[:, 0])

In [25]:
np.unique(trainY)

array([0, 1, 2, 3, 4, 5])

In [14]:
trainY

array([[5],
       [5],
       [5],
       ...,
       [5],
       [5],
       [5]])

In [15]:
np.unique(trainY)

array([0, 1, 2, 3, 4, 5])

In [16]:
print(num_samples)
print(n_classes)

72236
6
